In [ ]:
#ipynb 파일들은 구글 Colab에서 실습
import matplotlib.pyplot as plt

In [1]:
# 케라스(Keras)는 파이썬으로 작성된 오픈 소스 신경망 라이브러리
# 2017년 구글 텐서플로(Tensorflow)의 코어 라이브러리에 케라스를 포함하기로 결정
import tensorflow as tf
import tensorflow.keras.datasets as ds

ImportError: Traceback (most recent call last):
  File "c:\Users\YSB\OneDrive\Desktop\CV_01\.venv\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: DLL 초기화 루틴을 실행할 수 없습니다.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

# 데이터 셋

## MNIST(Modified National Institute of Standards and Technology) 데이터 셋

In [ ]:
(x_train, y_train), (x_test, y_test) = ds.mnist.load_data()

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
# 데이터 확인
print(x_train[0], y_train[0])

In [ ]:
plt.figure(figsize=(10,10))
plt.suptitle('MNIST', fontsize=30)
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i].reshape(28, 28), cmap=plt.cm.gray)
    plt.xlabel(str(y_train[i]))
plt.show()

## fashion-MNIST

In [ ]:
(fx_train, fy_train), (fx_test, fy_test) = ds.fashion_mnist.load_data()

In [ ]:
print(fx_train.shape, fy_train.shape, fx_test.shape, fy_test.shape)

In [ ]:
# 데이터 확인
print(fx_train[0], fy_train[0])

In [ ]:
fashion_class_names=['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
plt.figure(figsize=(10,10))
plt.suptitle('fashion-MNIST', fontsize=30)
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(fx_train[i].reshape(28, 28), cmap=plt.cm.gray)
    plt.xlabel(str(fy_train[i]) +'-'+fashion_class_names[fy_train[i]])
plt.show()

## CIFAR(Canadian Institute for Advanced Research)-10

In [ ]:
(cx_train, cy_train), (cx_test, cy_test) = ds.cifar10.load_data()

In [ ]:
print(cx_train.shape, cy_train.shape, cx_test.shape, cy_test.shape)

In [ ]:
print(cx_train[0], cy_train[0])

In [ ]:
cifar10_class_names=['airplane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship','truck']

In [ ]:
plt.figure(figsize=(10,10))
plt.suptitle('CIFAR-10',fontsize=30)
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(cx_train[i].reshape(32, 32, 3))
    plt.xlabel(str(cy_train[i][0])+'-'+cifar10_class_names[cy_train[i][0]])
plt.show()

# 학습데이터로 표현

## 이미지 자체를 그대로 표현


*   입력 이미지를 1차원 특징 벡터로, 0~1로 정규화로 표현
*   출력 라벨을 one-hot 인코딩 방법으로 표현



In [ ]:
# MNIST
#(x_train,y_train),(x_test,y_test) = ds.mnist.load_data()

print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

In [ ]:
# 학습 데이터 정규화
x_trainF = x_train.reshape(60000,784)   # 1차원 구조로 변경
x_testF = x_test.reshape(10000,784)

x_trainF = x_trainF.astype('float32')   # 0~1로 정규화하기 위해 실수형으로 변환
x_testF = x_testF.astype('float32')

x_trainF /= 255.0                       # 0~1로 정규화
x_testF /= 255.0

x_trainF.shape, x_testF.shape

In [ ]:
print(x_trainF[0])

In [ ]:
y_train10 = tf.keras.utils.to_categorical(y_train,10)   # one-hot 인코딩
y_test10 = tf.keras.utils.to_categorical(y_test,10)

y_train10.shape, y_test10.shape

In [ ]:
print(y_train[0], y_train10[0])

## 이미지 특징을 계산하여 사용


*   예를 들어 HOG는 81차원의 특징을 계산하여 사용



In [ ]:
import cv2
import numpy as np

imsize = 28 # size of image (28x28)
# HOG parameters:
winSize = (imsize, imsize) # 28, 28
blockSize = (imsize//2, imsize//2) # 14, 14
cellSize = (imsize//2, imsize//2) #14, 14
blockStride = (imsize//4, imsize//4) # 7, 7
nbins = 9
derivAperture = 1
winSigma = -1.0
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64
signedGradients = True

# define the HOG descriptor
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins, derivAperture, winSigma,
                        histogramNormType, L2HysThreshold, gammaCorrection, nlevels, signedGradients)

# compute HOG descriptors
x_train_hog = []
for i in range(x_train.shape[0]):
    descriptor = hog.compute(x_train[i]) # compute the HOG features
    x_train_hog.append(descriptor) # append it to the train decriptors list

x_test_hog = []
for i in range(x_test.shape[0]):
    descriptor = hog.compute(x_test[i]) # compute the HOG features
    x_test_hog.append(descriptor) # append it to the test descriptors list

#train_descriptors = np.array(train_descriptors)
x_train_hog = np.resize(x_train_hog, (x_train.shape[0], 81))

#test_descriptors = np.array(test_descriptors)
x_test_hog = np.resize(x_test_hog, (x_test.shape[0], 81))

In [ ]:
print(x_train_hog[0])

In [ ]:
y_train10 = tf.keras.utils.to_categorical(y_train,10)   # one-hot 인코딩
y_test10 = tf.keras.utils.to_categorical(y_test,10)

y_train10.shape, y_test10.shape

In [ ]:
print(y_train[0], y_train10[0])

# 데이터 증강

## 이미지처리 이용

In [ ]:
import cv2
import numpy as np

def rotate(img, angle_deg, scale=1.0):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle_deg, scale)
    return cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)

def translate(img, tx, ty):
    h, w = img.shape[:2]
    M = np.float32([[1,0,tx],[0,1,ty]])
    return cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)

def flip(img, mode=1):
    # mode: 1=좌우, 0=상하, -1=좌우+상하
    return cv2.flip(img, mode)

def scale_resize(img, fx=1.0, fy=1.0, keep_size=True):
    h, w = img.shape[:2]
    scaled = cv2.resize(img, (0,0), fx=fx, fy=fy, interpolation=cv2.INTER_LINEAR)
    if keep_size:
        # 중앙 크롭/패딩으로 원래 크기 유지
        nh, nw = scaled.shape[:2]
        out = np.zeros_like(img)
        if nh >= h and nw >= w:
            # 중앙 크롭
            ys = (nh - h)//2; xs = (nw - w)//2
            out = scaled[ys:ys+h, xs:xs+w].copy()
        else:
            # 패딩
            out[:] = 0
            ys = max((h - nh)//2, 0); xs = max((w - nw)//2, 0)
            out[ys:ys+nh, xs:xs+nw] = scaled
        return out
    return scaled

def blur(img, k=5):
    k = k if k % 2 == 1 else k+1
    return cv2.GaussianBlur(img, (k,k), 0)

In [ ]:
cols = 6
rows = 1
plt.figure(figsize=(10, 3))

img = cx_train[0]

# 원본
plt.subplot(rows, cols, 1)
plt.imshow(img); plt.title("Original"); plt.axis("off")

# 증강 결과
plt.subplot(rows, cols, 2)
plt.imshow(rotate(img, 20)); plt.title("Rotate 20"); plt.axis("off")

plt.subplot(rows, cols, 3)
plt.imshow(translate(img, 10, 10)); plt.title("Translate (10,10)"); plt.axis("off")

plt.subplot(rows, cols, 4)
plt.imshow(flip(img, 1)); plt.title("Flip"); plt.axis("off")

plt.subplot(rows, cols, 5)
plt.imshow(scale_resize(img, 1.2, 1.2)); plt.title("Scale * 1.2"); plt.axis("off")

plt.subplot(rows, cols, 6)
plt.imshow(blur(img)); plt.title("GaussianBlur"); plt.axis("off")

plt.show()

## Albumentations 라이브러리

In [ ]:
!pip install -q albumentations==1.4.4

In [ ]:
import albumentations as A

# 데이터 증강 파이프라인 정의
transform = A.Compose([
    A.HorizontalFlip(p=0.5),                      # 좌우 뒤집기
    A.VerticalFlip(p=0.2),                        # 상하 뒤집기
    A.RandomRotate90(p=0.3),                      # 90도 단위 회전
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=25, p=0.7),   # 이동, 확대/축소, 회전
    A.RandomBrightnessContrast(p=0.5),            # 밝기, 대비
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),  # 가우시안 노이즈
    A.HueSaturationValue(p=0.4),                  # 색상 변화
    A.Blur(blur_limit=3, p=0.3)                   # 블러
])

In [ ]:
img = cx_train[0]

fig, axes = plt.subplots(2, 4, figsize=(7,4))
axes = axes.ravel()

axes[0].imshow(img)
axes[0].set_title("Original")
axes[0].axis('off')

for i in range(1, 8):
    augmented = transform(image=img)    # 랜덤 데이터 증강을 수행, image는 증강할 이미지
    aug_img = augmented['image']        # 증강된 이미지
    axes[i].imshow(aug_img)
    axes[i].set_title(f"Aug {i}")
    axes[i].axis('off')

plt.show()